# Load Test deployed web application

This notebook pulls some images and tests them against the deployed web application. We submit requests asychronously 
which should reduce the contribution of latency.

In [ ]:
from urllib.parse import urlparse
import pandas as pd

from azure_utils.configuration.project_configuration import ProjectConfiguration
from azure_utils.machine_learning.utils import get_workspace_from_config
from azureml.core.webservice import AksWebservice

In [ ]:
ws = get_workspace_from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

Let's retrieve the web service.

In [ ]:
project_configuration = ProjectConfiguration("project.yml")
aks_service_name = project_configuration.get_settings('aks_service_name')
aks_service = AksWebservice(ws, name=aks_service_name)

We will test our service concurrently but only have 4 concurrent requests at any time. We have only deployed one pod 
on one node and increasing the number of concurrent calls does not really increase throughput. Feel free to try 
different values and see how the service responds.

In [ ]:
concurrent_requests = 4  # Number of requests at a time

Get the scoring URL and API key of the service.

In [ ]:
scoring_url = aks_service.scoring_uri
api_key = aks_service.get_keys()[0]

Below we are going to use [Locust](https://locust.io/) to load test our deployed model. First we need to write the 
locustfile.

In [ ]:
%%writefile locustfile.py
from locust import HttpLocust, TaskSet, task
import os
import pandas as pd
from utilities import text_to_json
from itertools import cycle

_NUMBER_OF_REQUESTS = os.getenv('NUMBER_OF_REQUESTS', 100)
dupes_test_path = './data_folder/dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
dupes_to_score = dupes_test.iloc[:_NUMBER_OF_REQUESTS, 4]
_SCORE_PATH = os.getenv('SCORE_PATH', "/score")
_API_KEY = os.getenv('API_KEY')


class UserBehavior(TaskSet):
    def on_start(self):
        print('Running setup')
        self._text_generator = cycle(dupes_to_score.apply(text_to_json))
        self._headers = {
            "content-type": "application/json",
            'Authorization': ('Bearer {}'.format(_API_KEY))
        }

    @task
    def score(self):
        self.client.post(_SCORE_PATH,
                         data=next(self._text_generator),
                         headers=self._headers)


class WebsiteUser(HttpLocust):
    task_set = UserBehavior
    # min and max time to wait before repeating task
    min_wait = 10
    max_wait = 200

Below we define the locust command we want to run. We are going to run at a hatch rate of 10 and the whole test will 
last 1 minute. Feel free to adjust the parameters below and see how the results differ. The results of the test will 
be saved to two csv files **modeltest_requests.csv** and **modeltest_distribution.csv**

In [ ]:
parsed_url = urlparse(scoring_url)
cmd = "locust -H {host} --no-web -c {users} -r {rate} -t {duration} --csv=modeltest --only-summary".format(
    host="{url.scheme}://{url.netloc}".format(url=parsed_url),
    users=concurrent_requests,  # concurrent users
    rate=10,  # hatch rate (users / second)
    duration='1m',  # test duration
)

In [ ]:
! API_KEY={api_key} SCORE_PATH={parsed_url.path} PYTHONPATH={os.path.abspath('../')} {cmd}

Here are the summary results of our test and below that the distribution infromation of those tests. 

In [ ]:
pd.read_csv("modeltest_requests.csv")

In [ ]:
pd.read_csv("modeltest_distribution.csv")

To tear down the cluster and all related resources go to the [tear down the cluster](07_TearDown.ipynb) notebook.